In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
from scipy.interpolate import RegularGridInterpolator

def integrate_particles(positions, concentrations, velocity_field, X, Y, dt, num_steps, L, phase_field, gamma):
    num_particles = len(positions)
    positions_over_time,  concentrations_over_time = np.zeros((num_steps+1, num_particles, 2)), np.zeros((num_steps+1, num_particles, 1))
    positions_over_time[0] = positions
    concentrations_over_time[0] = concentrations
    psiy, psix = -1*diff_wrt_y(velocity_field), diff_wrt_x(velocity_field)
    phix, phiy = diff_wrt_x(phase_field), diff_wrt_y(phase_field)
    polynomial_u, polynomial_v = (1-gamma)*interpolate(psiy)+gamma*interpolate(phix), (1-gamma)*interpolate(psix)+gamma*interpolate(phiy)
    t_span = (0, num_steps * dt)
    t_eval = [i * dt for i in range(num_steps+1)]
    t = [i*dt for i in range(num_steps+1)]
    du, dv = diff_wrt_x(psiy), diff_wrt_y(psix)
    polynomial_du, polynomial_dv = interpolate(du), interpolate(dv) 
    answer = new_vector_rk4(fun, t, polynomial_u, polynomial_v, positions_over_time, polynomial_du, polynomial_dv, concentrations_over_time, num_particles)
    return answer


def diff_wrt_y(original_matrix): 
    nt, nx, ny = original_matrix.shape
    umatrix = np.copy(original_matrix)
    for i in range(nt):
        umatrix[i] = np.gradient(original_matrix[i], axis=1) 
    return umatrix

def diff_wrt_x(original_matrix):
    vmatrix = np.copy(original_matrix)
    nt, nx, ny = original_matrix.shape
    for i in range(nt):
        vmatrix[i] = np.gradient(original_matrix[i], axis=0) 
    return vmatrix

def interpolate(umatrix):

    t = umatrix.shape[0]
    time = [i for i in range(t)]
    interpolator = RegularGridInterpolator((time, X, Y), umatrix, method = "cubic")
    return interpolator

def new_vector_rk4(f, t, polynomial_u, polynomial_v, positions_over_time, polynomial_du, polynomial_dv, concentrations_over_time, num_particles):
    n = len(t)
    y = np.concatenate((positions_over_time, concentrations_over_time), axis=2)

    for i in range(n - 1):
        h = t[i+1] - t[i]
        k1 = f(t[i], y[i], polynomial_u, polynomial_v, polynomial_du, polynomial_dv, num_particles)
        k2 = f(t[i] + 0.5*h, y[i] + 0.5*h*k1, polynomial_u, polynomial_v, polynomial_du, polynomial_dv, num_particles)
        k3 = f(t[i] + 0.5*h, y[i] + 0.5*h*k2, polynomial_u, polynomial_v, polynomial_du, polynomial_dv, num_particles)
        k4 = f(t[i+1], y[i] + h*k3, polynomial_u, polynomial_v, polynomial_du, polynomial_dv, num_particles)
        y[i+1] = y[i] + (h/6)*(k1 + 2*k2 + 2*k3 + k4)
    return np.mod(y, L)

def fun(t, y, polynomial_u, polynomial_v, polynomial_du, polynomial_dv, num_particles):
    xlist, ylist, conclist = [(i+L)% (2*L) -L for i in y[:, 0]], [(i+L)% (2*L) -L for i in y[:, 1]], [(i+L)% (2*L) -L for i in y[:, 2]] 
    timecoords = t*np.ones(num_particles)
    coords = [[x, y, z] for x, y, z in zip(timecoords, xlist, ylist)]
    dxdt, dydt = polynomial_u(coords), polynomial_v(coords)
    dxdt, dydt = np.array(dxdt).reshape(-1, 1), np.array(dydt).reshape(-1, 1)
    dudx, dvdy = polynomial_du(coords), polynomial_dv(coords)
    column = np.array(dvdy+dudx).reshape(-1, 1)
    c = np.array(conclist).reshape(-1, 1)
    dcdt = -1*c*(column)
    result = np.concatenate((dxdt, dydt), axis=1)
    result = np.concatenate((result, dcdt), axis=1)
    return result
